In [102]:
import math, time
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
import random
# from nmraNew import *

In [103]:
#constants

In [104]:
no_of_movements = 5

## Sensor Class

In [105]:
class sensor:
    def __init__(self, type, x, y, x_e = 0, y_e = 0):
        self.type = type
        self.x = x
        self.y = y
        self.x_e = x_e
        self.y_e = y_e

### Optimization function

In [106]:
def Fun(x,y, anchors, d):
    f = 0
    for i in range(3):
        # print(anchors[i].x,anchors[i].y )
        f +=  math.pow(math.sqrt(math.pow((x-anchors[i][0]),2) + math.pow((y-anchors[i][1]),2)) - d,2)
        # print(f)
    return f/3

## NMRA

In [107]:
def NMRA(x_lb,x_ub,y_lb,y_ub,maxIter,best,n,anchors):
    rats=[]
    workers=n-n//5
    breeders=n//5
    fitness=[]
    bp=0.5
    d = (x_ub-x_lb)/2
    for i in range(n):
        x1=x_lb+np.random.random()*(x_ub-x_lb)
        y1=y_lb+np.random.random()*(y_ub-y_lb)
        rats.append([x1,y1])
        fitness.append(Fun(x1,y1,anchors,d))
    I=fitness.index(min(fitness))
    ratBest=rats[I]
    # print(fitness)
    
    best=fitness[I]
    sorted_fitness = []
    for i in range(n):
        l = [fitness[i],i]
        sorted_fitness.append(l)

    changes=[]
    sorted_fitness.sort()
    # print(sorted_fitness)
    for itr in range(maxIter):
        S = rats
        tmp=rats
        # print(rats)
        
        # Workers phase
        for i in range(breeders, n):
            ab = []
            for j in range(breeders, n):
                ab.append(sorted_fitness[j][1])
            idx = sorted_fitness[i][1]
            lmda = np.random.random()
            random.shuffle(ab)
            S[idx][0] = S[idx][0] + lmda * (S[ab[0]][0] - S[ab[1]][0])
            S[idx][1] = S[idx][1] + lmda * (S[ab[0]][1] - S[ab[1]][1])
            # Calculating and updating new fitness and rat's coordinates
            fnew = Fun(S[idx][0], S[idx][1], anchors, d)
            if fnew < fitness[idx]:
                fitness[idx] = fnew
                rats[idx]= S[idx]
        
        # Breeders phase
        for i in range(breeders):
            if np.random.random() > bp:
                lmda = np.random.random()
                idx = sorted_fitness[i][1]
                
                S[idx][0] = (1-lmda)*S[idx][0] + lmda*(ratBest[0] - S[idx][0])
                S[idx][1] = (1-lmda)*S[idx][1] + lmda*(ratBest[1] - S[idx][1])
                fnew = Fun(S[idx][0], S[idx][1], anchors, d)
                if fnew < fitness[idx]:
                    fitness[idx] = fnew
                    rats[idx]= S[idx]
        
        


        # updating the sorted_fitness list
        sorted_fitness = []
        for i in range(n):
            l = [fitness[i],i]
            sorted_fitness.append(l)
        sorted_fitness.sort()
    
    idr=sorted_fitness[0][1]
        
    
    return rats[idr]

### Function to find virtual anchors

In [108]:
def find_virtual_anchors(x,y,d):
    anchors=[]
    anchors.append([x+d*np.cos(np.pi/3),y+d*np.sin(np.pi/3)])
    anchors.append([x+d,y])
    anchors.append([x-d,y]);
    anchors.append([x+d*np.cos(np.pi/3),y-d*np.sin(np.pi/3)]);
    anchors.append([x-d*np.cos(np.pi/3),y-d*np.sin(np.pi/3)]);
    anchors.append([x-d*np.cos(np.pi/3),y+d*np.sin(np.pi/3)]);
    return anchors

In [109]:
anchor1 = sensor('anchor',7.5,7.5)

In [110]:
# finding virtual anchors
virtual_anchors=find_virtual_anchors(anchor1.x,anchor1.y,4.5)
virtual_anchors=np.array(virtual_anchors)
x,y=virtual_anchors.T

In [111]:
x_target=np.random.uniform(0,15,5)
y_target=np.random.uniform(0,15,5)

### Plotting and localization

In [112]:
for i in range(no_of_movements):
    x_target[i]+=0.1*random.randint(-5,5)
    y_target[i]+=0.1*random.randint(-5,5)
    if x_target[i]<0 or x_target[i]>15:
        x_target[i]=random.randint(0,15)
    if y_target[i]<0 or y_target[i]>15:
        y_target[i]=random.randint(0,15)
    
    coordinates = []
    
    for i in range(5):
        d=math.dist([x_target[i],y_target[i]],[anchor1.x,anchor1.y]);
        virtual_anchors=find_virtual_anchors(anchor1.x,anchor1.y,d)
        va_dist=[]
        for j in range(len(virtual_anchors)):
            dist=math.dist([x_target[i],y_target[i]],[virtual_anchors[j][0],virtual_anchors[j][1]])
            l=[dist,j]
            va_dist.append(l)
        va_dist.sort()
        idx1,idx2=va_dist[0][1],va_dist[1][1]
        va1,va2=virtual_anchors[idx1],virtual_anchors[idx2]
        
        centeroid_x,centeroid_y=(va1[0]+va2[0]+anchor1.x)/3,(va1[1]+va2[1]+anchor1.y)/3
        
        coordinates.append(NMRA(centeroid_x-d,centeroid_x+d,centeroid_y-d,centeroid_y+d,50,3,100,[[anchor1.x,anchor1.y],va1,va2]))
#     print(coordinates,[x_target[i],y_target[i]])
    coordinates = np.array(coordinates)
    estimated_x, estimated_y = coordinates.T
    print(estimated_x, estimated_y)
    
    plt.scatter(x_target,y_target)
    plt.scatter(estimated_x, estimated_y)
    plt.show()

[ 9.56688061e-15 -2.51592826e-09  1.86168072e-06  8.16820072e-16
  3.89878726e-07] [ 1.92505245e-15 -1.73685421e-09  8.22878996e-07 -2.46807789e-16
  7.97077705e-07]
[ 1.64293093e-04  2.08176027e-13  1.86514534e-11 -1.01749634e-08
  2.03504959e-04] [ 3.54462336e-05 -1.97371884e-14 -1.52737360e-11 -4.87217368e-09
 -1.91560323e-05]
[-4.50402528e-12 -4.81155997e-09  8.03886068e-10  1.60875904e-11
  1.66465405e-05] [4.31926425e-12 3.92797855e-10 1.48848104e-09 4.42492994e-11
 9.68113613e-06]
[ 1.73058439e-07 -1.38255565e-07  1.31252367e-09  1.70935007e-08
 -1.75722401e-06] [ 2.80021879e-08  5.76820307e-08  2.46452302e-09 -7.33753745e-09
  1.15756404e-07]
[1.48012336e-11 2.09288795e-09 2.71099420e-06 3.30856076e-10
 1.30056128e-08] [4.75104554e-11 4.20475113e-09 2.28173617e-06 8.22437929e-10
 2.33059304e-08]


In [113]:
# # plotting graph for virtual and target nodes
# # %matplotlib notebook

# fig,ax=plt.subplots()
# scatter=ax.scatter(x,y)
# ax.set_xlim([0,15])
# ax.set_ylim([0,15])
# def update(itr):
#     global x_target
#     global y_target,x,y
    
#     # Adding random movement to the target nodes
#     for i in range(5):
#         x_target[i]+=0.1*random.randint(-5,5)
#         y_target[i]+=0.1*random.randint(-5,5)
#         if x_target[i]<0 or x_target[i]>15:
#             x_target[i]=random.randint(0,15)
#         if y_target[i]<0 or y_target[i]>15:
#             y_target[i]=random.randint(0,15)

#     for i in range(5):
#         d=math.dist([x_target[i],y_target[i]],[anchor1.x,anchor1.y]);
#         virtual_anchors=find_virtual_anchors(anchor1.x,anchor1.y,d)
#         va_dist=[]
#         for j in range(len(virtual_anchors)):
#             dist=math.dist([x_target[i],y_target[i]],[virtual_anchors[j][0],virtual_anchors[j][1]])
#             l=[dist,j]
#             va_dist.append(l)
#         va_dist.sort()
#         idx1,idx2=va_dist[0][1],va_dist[1][1]
#         va1,va2=virtual_anchors[idx1],virtual_anchors[idx2]
#         centeroid_x,centeroid_y=(va1[0]+va2[0]+anchor1.x)/3,(va1[1]+va2[1]+anchor1.y)/2
#         coordiantes=NMRA(centeroid_x-d,centeroid_x+d,centeroid_y-d,centeroid_y+d,50,3,100,[[anchor1.x,anchor1.y],va1,va2])
#         print(coordiantes,[x_target[i],y_target[i]])
#     plt.cla()
#     ax.set_xlim([0,15])
#     ax.set_ylim([0,15])
#     ax.scatter([7.5],[7.5],c='yellow')
#     ax.scatter(x,y)
#     ax.scatter(x_target,y_target,marker='^')
# ani=FuncAnimation(fig=fig,func=update,interval=1000)    
# plt.show()